<a href="https://colab.research.google.com/github/Rineeth04/amazon-reviews/blob/main/Amazon_reviews_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

   **<center><h1>Amazon Reviews sentiment analysis by Rineeth rao venepally</h1></center>**



In [ ]:
# imports
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import json
import nltk
import spacy
import string
import unicodedata
from bs4 import BeautifulSoup
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer

from IPython import display
display.set_matplotlib_formats('svg')
warnings.filterwarnings('ignore')

In [ ]:
# load the data
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/amazon_alexa.csv")
data.head()

,reviews,sentiment
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Here, we are going to use only 2 columns Independent(reviewText) and Dependent(rating). Let's ignore all other columns.  

In [ ]:
# selection columns!
data = data[['reviews', 'sentiment']]
data.head()

,reviews,sentiment
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


In [ ]:
# check the shape
data.shape

(3150, 2)

In [ ]:
# check whehter it's having null values or not
data.isnull().sum()

reviews      0
sentiment    0
dtype: int64

In [ ]:
# check unique categories
data['sentiment'].value_counts()

1    2893
0     257
Name: sentiment, dtype: int64

In [ ]:
# if rating is above 3 we will consider it as 1 else 0.
data["sentiment"] = data["sentiment"].apply(lambda x: 1 if x < 3  else 0) # positive as 0 and negative as 1

### Visualize words using WordCloud

In [ ]:
data['sentiment'].value_counts()

1    3150
Name: sentiment, dtype: int64

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image

# get the text based on ratings!
rating_zero = data['reviews'][data['sentiment'] == 0]
rating_one = data['reviews'][data['sentiment'] == 1]


# word tokenize the data
def list_tokenizer(rating):
    ratings = " ".join(rating)
    ratings = nltk.word_tokenize(ratings)
    return str(ratings)

# visualizer
def word_cloud(rating, number):
    wc = WordCloud(background_color = 'black', max_font_size = 50, max_words = 100)
    wc.generate(rating)
    plt.figure(figsize=(10,8))
    plt.imshow(wc, interpolation = 'bilinear')
    plt.title(f'WordCloud for {number}')
    plt.axis('off');

In [ ]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Now, we have suitable columns for model building, but before that we need to pre-process the text. Let's do that!

### Pre-processing!
If you want a quick recap on pre-processing techniques you can check the notebook [**here**](https://github.com/RAravindDS/Learny/blob/main/NLP/TextCleaning/Cleaning_text_data.ipynb)

In [ ]:
# 1. lowering case
data['reviews'] = data['reviews'].str.lower()  # lowering the case!
data.head()

,reviews,sentiment
0,love my echo!,1
1,loved it!,1
2,"sometimes while playing a game, you can answer...",1
3,i have had a lot of fun with this thing. my 4 ...,1
4,music,1


In [ ]:
# 2. remove punctuation!
data['reviews'] = data['reviews'].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+', '', x))  # it removes the punctuation
data.head()

,reviews,sentiment
0,love my echo,1
1,loved it,1
2,sometimes while playing a game you can answer ...,1
3,i have had a lot of fun with this thing my 4 y...,1
4,music,1


In [ ]:
# 3. remove stopwords! (you can check the notebook, if you have any doubt)

from spacy.lang.en.stop_words import STOP_WORDS
data['reviews'] = data['reviews'].apply(lambda x: " ".join([i for i in x.split() if i not in STOP_WORDS]) )

data.head()

,reviews,sentiment
0,love echo,1
1,loved,1
2,playing game answer question correctly alexa s...,1
3,lot fun thing 4 yr old learns dinosaurs contro...,1
4,music,1


In [ ]:
# 4. remove url and tags!
data['reviews'] = data['reviews'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))

data.head()

,reviews,sentiment
0,love echo,1
1,loved,1
2,playing game answer question correctly alexa s...,1
3,lot fun thing 4 yr old learns dinosaurs contro...,1
4,music,1


In [ ]:
# 5. remove html tags!
data['reviews'] = data['reviews'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

data.head()

,reviews,sentiment
0,love echo,1
1,loved,1
2,playing game answer question correctly alexa s...,1
3,lot fun thing 4 yr old learns dinosaurs contro...,1
4,music,1


In [ ]:
# 7. remove emails
data['reviews'] = data['reviews'].apply(lambda x: re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+\b)', '', x))  # it will remove te emails

In [ ]:
# 8. remove extra spaces
data['reviews'] = data['reviews'].apply(lambda x: " ".join(x.split()))
data.head()

,reviews,sentiment
0,love echo,1
1,loved,1
2,playing game answer question correctly alexa s...,1
3,lot fun thing 4 yr old learns dinosaurs contro...,1
4,music,1


In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# 9. lemmatizer

%time
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

data["reviews"] = data["reviews"].apply(lambda text: lemmatize_words(text))
data.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


,reviews,sentiment
0,love echo,1
1,loved,1
2,playing game answer question correctly alexa s...,1
3,lot fun thing 4 yr old learns dinosaur control...,1
4,music,1


### Text to Words

Now, we have pre-processed the file, let's start the model buidling.

But before model building, we need to convert the text to numbers. So, let's do this by two methods `BOW` & `TF-IDF`~


#### 1. Bag of Words
If you have any doubt check our [**notebook**](https://github.com/RAravindDS/Learny/blob/main/NLP/BagofWords.ipynb)

But before this, let's split the data!

In [ ]:
# data split
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(data['reviews'], data['sentiment'], test_size = 0.3)

In [ ]:
# Convert text to numbers using (BOW)!
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# let's convert
xtrain_bow = vectorizer.fit_transform(xtrain).toarray()
xtest_bow = vectorizer.transform(xtest).toarray()


#### 2. TF-IDF

If you have any doubt you can check our [**notebook**](https://github.com/RAravindDS/Learny/blob/main/NLP/TFIDF/TFIDF.ipynb)

In [ ]:
# Convert text to numbers using (TF-IDF)
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vectorizer = TfidfVectorizer()

# let's convert
xtrain_tf = tf_vectorizer.fit_transform(xtrain).toarray()
xtest_tf = tf_vectorizer.transform(xtest).toarray()

### Model Building

Here, we are going to use the `Gaussina NB` model.

First we will see the results for `BOW`

In [ ]:
from sklearn.naive_bayes import GaussianNB

# USING BOW
clf_bow = GaussianNB().fit(xtrain_bow, ytrain)  # fitting
prediction_bow = clf_bow.predict(xtest_bow)  # predictions

# USINGN TFIDF
clf_tf = GaussianNB().fit(xtrain_tf, ytrain)
prediction_tf = clf_tf.predict(xtest_tf)

# ***Building KNN  model***

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(xtrain_bow,ytrain)
pred_train = knn.predict(xtrain_bow)
pred_test = knn.predict(xtest_bow)

# ***Building Logistic regression model***

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
# let's check our model!
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def metrics(prediction, actual):
    print('Confusion_matrix \n', confusion_matrix(actual, prediction))
    print('\nAccuracy:', accuracy_score(actual, prediction))
    print('\nclassification_report\n')
    print(classification_report(actual, prediction))


metrics(prediction_bow, ytest)

Confusion_matrix 
 [[945]]

Accuracy: 1.0

classification_report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       945

    accuracy                           1.00       945
   macro avg       1.00      1.00      1.00       945
weighted avg       1.00      1.00      1.00       945



In [ ]:
metrics(prediction_tf, ytest)

Confusion_matrix 
 [[945]]

Accuracy: 1.0

classification_report

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       945

    accuracy                           1.00       945
   macro avg       1.00      1.00      1.00       945
weighted avg       1.00      1.00      1.00       945

